# Libs

In [1]:
import numpy as np
import pandas as pd

from cloudmanufacturing.data import read_fatahi_dataset
from cloudmanufacturing.solvers.naive_solver import NaiveSolver

/Users/fomindmitry/miniconda3/envs/graph/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Loading data

In [2]:
dataset = read_fatahi_dataset("../data/fatahi.xlsx")

  0%|          | 0/18 [00:00<?, ?it/s]

100%|██████████| 18/18 [00:03<00:00,  5.21it/s]


# Solving problem

In [3]:
naive_solver = NaiveSolver(dataset)
info_problem = naive_solver.solve_problem(0)
info_problem

{'path': {'suboperation_0': array([0., 3., 0., 0., 3.]),
  'suboperation_1': array([9., 9., 9., 9.]),
  'suboperation_2': array([9., 9., 5., 5., 6., 6.]),
  'suboperation_3': array([3., 0., 3., 3., 0., 3., 3.]),
  'suboperation_4': array([9., 9., 5., 5., 7., 7.])},
 'cost': 6325.046463556215}

In [4]:
info = naive_solver.solve_all()
print("total_cost:", naive_solver.total_cost)
info

total_cost: 239778.25824428024


{'problem_0': {'path': {'suboperation_0': array([0., 3., 0., 0., 3.]),
   'suboperation_1': array([9., 9., 9., 9.]),
   'suboperation_2': array([9., 9., 5., 5., 6., 6.]),
   'suboperation_3': array([3., 0., 3., 3., 0., 3., 3.]),
   'suboperation_4': array([9., 9., 5., 5., 7., 7.])},
  'cost': 6325.046463556215},
 'problem_1': {'path': {'suboperation_0': array([9., 9., 9., 5., 5., 7., 7., 6., 6.]),
   'suboperation_1': array([9., 9., 9., 5., 5., 7., 7., 6., 6.]),
   'suboperation_2': array([0., 3., 0., 3., 6.]),
   'suboperation_3': array([9., 9., 9., 5., 5., 7., 6.]),
   'suboperation_4': array([0., 3., 0., 3., 6., 6.])},
  'cost': 10974.04987662026},
 'problem_2': {'path': {'suboperation_0': array([9., 9., 9., 9., 5., 5., 7., 7., 6., 6.]),
   'suboperation_1': array([9., 9., 9., 9., 5., 5., 5.]),
   'suboperation_2': array([9., 9., 9., 9., 5., 5., 7., 7., 6., 6.]),
   'suboperation_3': array([9., 9., 9., 9., 5., 5., 5., 6.]),
   'suboperation_4': array([3., 3., 3., 0., 0., 6.])},
  'c

In [5]:
def softmax(x):
    reciprocal_x = 1 / x
    exp_reciprocal_x = np.exp(reciprocal_x - np.max(reciprocal_x))
    return exp_reciprocal_x / np.sum(exp_reciprocal_x)

In [96]:
import numpy as np


class NaiveSolver_plus:
    def __init__(self, dataset, n_bests=1, step_forward=1, skip_step=1):
        self.problems = dataset
        self.total_cost = 0
        self.n_best = n_bests
        self.step_forward = step_forward
        self.cost_operations = None
        self.trans_cost = None
        self.skip_step = skip_step

    def look_forward(self, horizon, start_city=None, cost=0):
        visited_cities = []
        for stage in horizon:
            if start_city is None:
                reverse_probabilities = 1 / self.cost_operations[stage]
                reverse_probabilities /= np.sum(reverse_probabilities)

                city = np.random.choice(
                    np.argsort(self.cost_operations[stage]), p=reverse_probabilities
                )

                visited_cities.append(city)
                cost += self.cost_operations[stage][city]
            else:
                cost_total = self.cost_operations[stage] + self.trans_cost[start_city]
                variants = np.argsort(cost_total)[: self.n_best]

                reverse_probabilities = 1 / cost_total[variants]
                reverse_probabilities /= np.sum(reverse_probabilities)

                city = np.random.choice(variants, p=reverse_probabilities)

                visited_cities.append(city)
                cost += cost_total[city]
        return cost, visited_cities

    def sample_forward(self, horizon, init_city, num):
        rand_walk = [self.look_forward(horizon, init_city) for _ in range(num)]
        return rand_walk

    def solve_suboperaion(self, available_operations):
        """
        Solve problem for one suboperation
        """
        sub_problem_data = []
        city = None
        skip_list = []
        for i, stage in enumerate(available_operations):
            horizon = available_operations[i : i + self.step_forward]

            if len(skip_list) == 0:
                rand_walk = self.sample_forward(horizon, city, 500)
                min_index = np.argmin([item[0] for item in rand_walk])
                skip_list = rand_walk[min_index][1][: self.skip_step]

            if city is None:
                # city = rand_walk[min_index][1][0]
                city = skip_list.pop(0)
                sub_problem_data.append([city, self.cost_operations[stage, city]])
            else:
                # new_city = rand_walk[min_index][1][0]
                new_city = skip_list.pop(0)
                cost_total = self.cost_operations[stage] + self.trans_cost[city]
                city = new_city
                sub_problem_data.append([city, cost_total[city]])

        return np.array(sub_problem_data)[:, 0], np.sum(
            np.array(sub_problem_data)[:, 1]
        )

    def solve_problem(self, num_problem, save_costs=False):
        """
        Solve operation
        """
        n_tasks = self.problems[num_problem]["n_tasks"]
        operation = self.problems[num_problem]["operation"]
        dist = self.problems[num_problem]["dist"]
        time_cost = self.problems[num_problem]["time_cost"]
        op_cost = self.problems[num_problem]["op_cost"]
        productivity = self.problems[num_problem]["productivity"]
        transportation_cost = self.problems[num_problem]["transportation_cost"]

        self.cost_operations = time_cost * op_cost / productivity
        self.trans_cost = dist * transportation_cost

        problem_cost = 0
        problem_path = {}

        for n_sub in range(n_tasks):
            available_operations = np.nonzero(operation[:, n_sub])[0]
            path, cost = self.solve_suboperaion(available_operations)

            problem_cost += cost
            problem_path[f"suboperation_{n_sub}"] = path
        if save_costs:
            self.total_cost += problem_cost
        return {"path": problem_path, "cost": problem_cost}

In [103]:
naive_solver_plus = NaiveSolver_plus(dataset, n_bests=3, step_forward=3, skip_step=1)

In [105]:
naive_solver_plus.solve_problem(0)

{'path': {'suboperation_0': array([0., 3., 0., 0., 3.]),
  'suboperation_1': array([1., 1., 1., 1.]),
  'suboperation_2': array([1., 9., 5., 5., 6., 6.]),
  'suboperation_3': array([2., 0., 3., 3., 0., 3., 3.]),
  'suboperation_4': array([1., 9., 5., 5., 7., 7.])},
 'cost': 7138.936050572123}

In [9]:
naive_solver = NaiveSolver(dataset)
info_problem = naive_solver.solve_problem(0)
info_problem

{'path': {'suboperation_0': array([0., 3., 0., 0., 3.]),
  'suboperation_1': array([9., 9., 9., 9.]),
  'suboperation_2': array([9., 9., 5., 5., 6., 6.]),
  'suboperation_3': array([3., 0., 3., 3., 0., 3., 3.]),
  'suboperation_4': array([9., 9., 5., 5., 7., 7.])},
 'cost': 6325.046463556215}

In [10]:
n_tasks = dataset[0]["n_tasks"]
operation = dataset[0]["operation"]
dist = dataset[0]["dist"]
time_cost = dataset[0]["time_cost"]
op_cost = dataset[0]["op_cost"]
productivity = dataset[0]["productivity"]
transportation_cost = dataset[0]["transportation_cost"]

# Create cost matrices
cost_operations = time_cost * op_cost / productivity
trans_cost = dist * transportation_cost

In [11]:
trans_cost[0] + cost_operations[6]

array([   183.69096349,    750.51627815,    838.78758119,    483.14460627,
          639.96857167, 135989.59530569,    737.59866504,    227.40822959,
          776.1957413 ,   1031.29260476])